# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是一名苹果官方旗舰店的顶级销售员，现在为客人答疑，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
1.
[客户问题]：这款iPhone的最大特点是什么？
[销售回答]：这款iPhone搭载了最新的A系列芯片，性能非常强大，同时拍照效果也非常出色，能让您的生活和工作更加便捷。

2.
[客户问题]：iPhone的电池续航怎么样？
[销售回答]：我们的iPhone在正常使用情况下，电池续航可以达到一天以上，而且我们有智能省电技术，能有效延长电池使用寿命。

3.
[客户问题]：iPhone的存储容量有哪些选择？
[销售回答]：iPhone提供多种存储容量选择，从128GB到1TB不等，您可以根据自己的需求进行选择。

4.
[客户问题]：MacBook Air和MacBook Pro有什么区别？
[销售回答]：MacBook Air更轻薄便携，适合日常办公和轻度使用；而MacBook Pro性能更强，适合专业设计和视频剪辑等高性能需求。

5.
[客户问题]：iPad适合办公吗？
[销售回答]：当然适合，iPad搭配智能键盘和Apple Pencil，可以实现高效办公，而且携带方便，是商务人士的理想选择。

6.
[客户问题]：Apple Watch有哪些健康功能？
[销售回答]：Apple Watch具有心率监测、睡眠追踪、健身记录等多种健康功能，可以帮助您更好地关注自己的身体状况。

7.
[客户问题]：AirPods和AirPods Pro有什么区别？
[销售回答]：AirPods Pro相比AirPods，增加了主动降噪功能、透明模式和自定义贴合的硅胶耳套，佩戴更舒适，音质更佳。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](apple_sales_data.txt) 文件中

In [1]:
#你是一名苹果官方旗舰店的顶级销售员
with open("apple_sales_data.txt", encoding="UTF-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

Created a chunk of size 111, which is longer than the specified 100
Created a chunk of size 101, which is longer than the specified 100


In [5]:
docs[0]

Document(page_content='[客户问题]：这款iPhone的最大特点是什么？\n[销售回答]：这款iPhone搭载了最新的A系列芯片，性能非常强大，同时拍照效果也非常出色，能让您的生活和工作更加便捷。')

In [6]:
len(docs)

20

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

D:\Miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "这款iPhone的最大特点是什么"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：这款iPhone的最大特点是什么？
[销售回答]：这款iPhone搭载了最新的A系列芯片，性能非常强大，同时拍照效果也非常出色，能让您的生活和工作更加便捷。

[客户问题]：iPhone的屏幕尺寸有哪些？
[销售回答]：iPhone提供多种屏幕尺寸选择，从小屏的iPhone SE到大屏的iPhone 13 Pro Max，满足不同用户的需求。

[客户问题]：iPhone的存储容量有哪些选择？
[销售回答]：iPhone提供多种存储容量选择，从128GB到1TB不等，您可以根据自己的需求进行选择。

[客户问题]：iPhone的摄像头像素是多少？
[销售回答]：我们的iPhone摄像头像素会根据不同型号有所差异，但都采用了先进的感光元件和图像处理技术，确保拍照效果清晰。



In [11]:
db.save_local("apple_customer_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [15]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [16]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002C2BA80E020>, search_kwargs={'k': 3})

In [17]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\Miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 你们会提供家具吗？
[销售回答] 我们的精装房会提供基础家具和家电，让您拎包入住。

[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

[客户问题] 都有哪些户型？
[销售回答] 我们有从一室到四室不等的多种户型，定能满足您不同的居住需求。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [18]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [19]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。



### 提取向量数据库中的`销售回答`

In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

#### 尝试各种问题

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想离医院近点"

print(sales(query))

[]


/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [28]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: []

score:0.75 ans: []



/root/miniconda3/envs/langchain/lib/python3.10/site-packages/langchain/schema/vectorstore.py:287: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。', '这个区域正在快速发展，未来的升值潜力非常大。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [29]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [31]:
qa_chain({"query": "你们小区有200万的房子吗？"})

{'query': '你们小区有200万的房子吗？',
 'result': '对不起，我无法回答这个问题，因为我是一个人工智能，没有实时的房地产信息。建议你直接联系房地产经纪人或者查阅相关房地产网站获取信息。'}

In [32]:
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [33]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


## 加载 FAISS 向量数据库已有结果

In [34]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [35]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [36]:
qa_chain({"query": "我想买别墅，你们有么"})

{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供这样的服务。我是一个人工智能，我可以帮助回答问题，但我不能出售商品或房产。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [38]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [39]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [40]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [41]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}